In [31]:
import numpy as np
import csv
import predict

In [35]:
def tanh_prime(x):
    return 1.0-np.square(np.tanh(x))

class NeuralNetwork:
    def __init__(self):
#         load dataset 
        reader=csv.reader(open('self_normalised_data.csv','r'),delimiter=',')
        x=list(reader)
        features=np.array(x[2:]).astype('float')
        np.random.shuffle(features)
        
        # we save the dataset metadata for the prediction part of the network        
        self.predict=predict.Predict(float(x[0][0]),float(x[0][1]),float(x[0][2]),float(x[0][3]),float(x[0][4]),float(x[0][5]))
        
        # car attribute and price are splitted, note that 1 is appended at each car for the bias
        data_x=np.concatenate((features[:,:3],np.ones((features.shape[0],1))),axis=1)
        data_y=features[:,3:]
        
        # we set a threshold at 80% of the data
        self.m=features.shape[0]
        self.m_train_set=int(self.m*0.8)
        
        # split the train,test by the threshold
        self.x_train,self.x_test=data_x[:self.m_train_set,:],data_x[self.m_train_set:,:]
        self.y_train,self.y_test=data_y[:self.m_train_set,:],data_y[self.m_train_set:,:]
        
        # initialise the neural network parameters
        self.a2,self.z2,self.a3,self.z3,self.a4,self.z4=(None,)*6
        self.djdw1,self.djdw2,self.djdw3=(None,)*3
        self.delta2,self.delta3,self.delta4=(None,)*3
        self.gradient,self.numericalGradient=(None,)*2
        self.learning_rate=0.01
        self.Lambda=0.01
        
        # initialise weights
        self.w1=np.matrix([
         [0.01, 0.05, 0.07],
            [0.2, 0.041, 0.11],
            [0.04, 0.56, 0.13],
            [0.1, 0.1, 0.1] ])
        
        self.w2=np.matrix([
            [0.04, 0.78],
            [0.4, 0.45],
            [0.65, 0.23],
            [0.1, 0.1]])

        self.w3=np.matrix([
            [0.04],
            [0.41],
            [0.1]])
        
    def forward(self):
        
        #first layer
        self.z2=np.dot(self.x_train,self.w1)
        self.a2=np.tanh(self.z2)
        
        # adding bias to first layer
        ba2=np.ones((self.a2.shape[0],1))
        self.a2=np.concatenate((self.a2,ba2),axis=1)
        
        # second layer
        self.z3=np.dot(self.a2,self.w2)
        self.a3=np.tanh(self.z3)
        
        # adding bias to second layer
        ba3=np.ones((self.a3.shape[0],1))
        self.a3=np.concatenate((self.a3,ba3),axis=1)
        
        # output layer
        self.z4=np.dot(self.a3,self.w3)
        self.a4=np.tanh(self.z4)
        
    def backward(self):
        
        # gradient of the cost function with regards to W3
        self.delta4=np.multiply(-(self.y_train-self.a4),tanh_prime(self.z4))
        self.djdw3=np.dot(self.a3.T,self.delta4)/self.m_train_set+self.Lambda*self.w3
        
        # gradient of the cost function with regards to W2

        self.delta3= np.multiply(np.dot(self.delta4,self.w3.T),tanh_prime(np.concatenate((self.z3,np.ones((self.z3.shape[0],1))),axis=1)))
        self.djdw2=np.dot(self.a2.T,np.delete(self.delta3,2,axis=1))/self.m_train_set+self.Lambda*self.w2
        
        # gradient of the cost function with regards to W2
        self.delta2=np.multiply(np.dot(np.delete(self.delta3,2,axis=1),self.w2.T),tanh_prime(np.concatenate((self.z2,np.ones((self.z2.shape[0],1))),axis=1)))
        self.djdw1=np.dot(self.x_train.T,np.delete(self.delta2,3,axis=1))/self.m_train_set+self.Lambda*self.w1
        
    def upgrade_gradient(self):
        
        self.w1-=self.learning_rate*self.djdw1
        self.w2-=self.learning_rate*self.djdw2
        self.w3-=self.learning_rate*self.djdw3        
        
    def cost_function(self):
        
        return 0.5*np.sum(np.square(self.y_train-self.a4))+0.5*self.Lambda*(np.sum(np.square(self.w1))+
                                                                     np.sum(np.square(self.w2))+
                                                                     np.sum(np.square(self.w3)))

    def predict(self,X):
        
        self.x_train=X
        self.forward()
        return self.a4
 
    def summary(self, step):
        print("Iteration: %d, Loss %f" % (step, self.cost_function()))
        print("RMSE: " + str(np.sqrt(np.mean(np.square(self.a4 - self.y_train)))))
        print("MAE: " + str(np.sum(np.absolute(self.a4 - self.y_train)) / self.m_train_set))
#         print("R2: " + str(self.r2()))
        
    def predict_price(self,km,fuel,age):
        self.x_train=np.concatenate((self.predict.input(km,fuel,age),np.ones((1,1))),axis=1)
        nn.forward()
        print (self.predict.output(self.a4[0]))
    

    
        
nn=NeuralNetwork()

print('### Training data')
nb_it=1000
for step in range(nb_it):
    
    nn.forward()
    nn.backward()
    nn.upgrade_gradient()
    
    if step%100==0:
        nn.summary(step)

print('### Testing data')
nn.x_train=nn.x_test
nn.y_train=nn.y_test
nn.forward()

print('### Testing summary')
nn.summary(nb_it)

print('### Predict')
nn.predict_price(168000.0,'Diesel',5.0)

### Training data
Iteration: 0, Loss 155.203642
RMSE: 0.2069390172758966
MAE: 0.17417167118105595
Iteration: 100, Loss 88.069694
RMSE: 0.15588169577575856
MAE: 0.13015562660597488
Iteration: 200, Loss 57.985401
RMSE: 0.12648241322530754
MAE: 0.10380462445056017
Iteration: 300, Loss 41.997377
RMSE: 0.10763905928592811
MAE: 0.08637920236681644
Iteration: 400, Loss 32.543963
RMSE: 0.09475054318793462
MAE: 0.07412233590915976
Iteration: 500, Loss 26.517796
RMSE: 0.08552697308130733
MAE: 0.0651627636282171
Iteration: 600, Loss 22.459064
RMSE: 0.0787079761973521
MAE: 0.058454080127589826
Iteration: 700, Loss 19.611050
RMSE: 0.07354663532480987
MAE: 0.05337334505967323
Iteration: 800, Loss 17.549448
RMSE: 0.06957204088264873
MAE: 0.049467925648333466
Iteration: 900, Loss 16.020691
RMSE: 0.066471504290408
MAE: 0.046451780092812416
### Testing data
### Testing summary
Iteration: 1000, Loss 3.495916
RMSE: 0.062029003155891944
MAE: 0.010522228643733969
### Predict
[[74128.07826376]]
